<a href="https://colab.research.google.com/github/francescopandolfo/TESI_LM32/blob/main/RF%20-%20variazione_relativa%20-%20dati%20riordinati.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [98]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings("ignore")
pathDrive = 'drive/MyDrive/UNICT/_TESI/dati/'

df = pd.read_csv(pathDrive + 'dati_interpolati_diff_1d-2d_8gg - riordinati.csv', sep=';')

df['edam_ecne_eplu_diff'] = df['edam_ecne_eplu_diff'].str.replace(',','.').astype(float)#
df['edam_ecor_emcn_diff'] = df['edam_ecor_emcn_diff'].str.replace(',','.').astype(float)
df['edam_emsg_eplu_diff'] = df['edam_emsg_eplu_diff'].str.replace(',','.').astype(float)
df['edam_epdn_ecne_diff'] = df['edam_epdn_ecne_diff'].str.replace(',','.').astype(float)
df['edam_epdn_emcn_diff'] = df['edam_epdn_emcn_diff'].str.replace(',','.').astype(float)
df['eint_ecpn_emgl_diff'] = df['eint_ecpn_emgl_diff'].str.replace(',','.').astype(float)#
df['eint_esln_emgl_diff'] = df['eint_esln_emgl_diff'].str.replace(',','.').astype(float)
df['eint_esln_espc_diff'] = df['eint_esln_espc_diff'].str.replace(',','.').astype(float)
df['emeg_ecpn_emgl_diff'] = df['emeg_ecpn_emgl_diff'].str.replace(',','.').astype(float)
df['emfn_ecor_ecri_diff'] = df['emfn_ecor_ecri_diff'].str.replace(',','.').astype(float)
df['emfn_ecor_emcn_diff'] = df['emfn_ecor_emcn_diff'].str.replace(',','.').astype(float)
df['emfn_emcn_epdn_diff'] = df['emfn_emcn_epdn_diff'].str.replace(',','.').astype(float)
df['epdn_ecne_ecpn_diff'] = df['epdn_ecne_ecpn_diff'].str.replace(',','.').astype(float)
df['epdn_eint_ecpn_diff'] = df['epdn_eint_ecpn_diff'].str.replace(',','.').astype(float)##
df['epdn_eint_espc_diff'] = df['epdn_eint_espc_diff'].str.replace(',','.').astype(float)
df['epdn_emfn_espc_diff'] = df['epdn_emfn_espc_diff'].str.replace(',','.').astype(float)
df['eplu_ecne_ecpn_diff'] = df['eplu_ecne_ecpn_diff'].str.replace(',','.').astype(float)
df['eplu_emeg_ecpn_diff'] = df['eplu_emeg_ecpn_diff'].str.replace(',','.').astype(float)
df['eplu_emeg_emal_diff'] = df['eplu_emeg_emal_diff'].str.replace(',','.').astype(float)
df['eplu_emsg_emal_diff'] = df['eplu_emsg_emal_diff'].str.replace(',','.').astype(float)

#df = shuffle(df)
#_dftarget = df['evento'].astype(int).values

df_train = df[:126]
y_train = df_train['evento'].astype(int).values

df_val = df[126:170]
y_val = df_val['evento'].astype(int).values

df_test = df[170:]
y_test = df_test['evento'].astype(int).values



In [99]:
def toDataFrame(df):
  format = '%d/%m/%Y'
  df['date'] = pd.to_datetime(df['day'], format=format)
  df = df.set_index('date')
  df = df.drop(columns=['day'])
  df = df.drop(columns=['evento'])

  for i in reversed(range(len(df.columns))):
    if df.dtypes[df.columns[i]] == 'object':
      df = df.drop(df.columns[i], axis=1)

  return df

df_train = toDataFrame(df_train)
df_val = toDataFrame(df_val)
df_test = toDataFrame(df_test)

#normalized_df = (df - df.mean()) / df.std()
#df = normalized_df

In [ ]:
#print(df_test.info())

In [77]:
from sklearn import datasets, decomposition
pca_feature_number = 16
pca = decomposition.PCA(n_components = pca_feature_number)
pca.fit(df)
new_df = pca.transform(df)

for i in range(pca_feature_number):
  df['f_' + str(i)] = new_df[:,i]

df = df.filter(like='f_')
#print(df.info())

In [ ]:
#from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

#y = _dftarget
#split_at = 150

#max_f1_score = 0
#max_f1_score_who = ""
max_accuracy_train = 0
max_accuracy_val = 0
max_accuracy_test = 0

max_accuracy_train_who = ""
max_accuracy_val_who = ""
max_accuracy_test_who = ""
contatore = 0

clf = RandomForestClassifier(max_depth=5, random_state=0)

for i in range(pca_feature_number):
  for j in range(pca_feature_number):
    for a in range(pca_feature_number):
      for b in range(pca_feature_number):
        for c in range(pca_feature_number):
          for d in range(pca_feature_number):
            #for e in range(pca_feature_number):
            #  for f in range(pca_feature_number):
            #    for g in range(pca_feature_number):

                  #if i<j and j<a and a<b and b<c and c<d    and i==0 and j==1 and a==2 and b==3 and c==4 and d==5:
                  if i<j and j<a and a<b and b<c and c<d:
                    contatore += 1
                    who = str(contatore) + ": features " + str(i) + " - " + str(j) + " - " + str(a) + " - " + str(b) + " - " + str(c) + " - " + str(d)

                    X_train = df_train.values[:, [i,j,a,b,c,d]]
                    X_val   = df_val.values[:, [i,j,a,b,c,d]]
                    X_test  = df_test.values[:, [i,j,a,b,c,d]]

                    #X_train = X[:split_at]
                    ##X_test = X[split_at:]

                    #y_train = y[:split_at]
                    #y_test = y[split_at:]

                    ################ LOGISTIC REGRESSION with CROSS VALIDATION
                    #lr = LogisticRegressionCV(cv=5, random_state=0, class_weight='balanced') #scoring per cambiare metrica
                    #lr.fit(X_train, y_train)
                    #acc_s = lr.score(X_train, y_train)*100
                    #acc_s_test = lr.score(X_test, y_test)*100
                    ###########################################################

                    ################ RANDOM FOREST
                    clf.fit(X_train, y_train)
                    acc_train = clf.score(X_train,  y_train,  sample_weight=None)*100
                    acc_val   = clf.score(X_val,    y_val,    sample_weight=None)*100
                    #acc_test  = clf.score(X_test,   y_test,   sample_weight=None)*100
                    #acc_s = np.mean(cross_val_score(clf, X_train, y_train, cv=5))*100

                    ##############################

                    acc_tot = acc_train + acc_val

                    if max_accuracy_train < acc_train or max_accuracy_val < acc_val:
                      print( who + ";" + "{:.1f}".format(acc_train) + ";" + "{:.1f}".format(acc_val) +";" + "{:.1f}".format(acc_tot) )

                      if max_accuracy_train < acc_train:
                        max_accuracy_train = acc_train
                        max_accuracy_train_who = who

                      if max_accuracy_val < acc_val:
                        max_accuracy_val = acc_val
                        max_accuracy_val_who = who



#print( max_f1_score_who )
#print( max_accuracy_score_who )
print( "---------------------> " + max_accuracy_train_who + " " + str(max_accuracy_train) )
print( "---------------------> " + max_accuracy_val_who + " " + str(max_accuracy_val) )
#print( conf_mat )



1: features 0 - 1 - 2 - 3 - 4 - 5;91.3;54.5;145.8
2: features 0 - 1 - 2 - 3 - 4 - 6;96.0;54.5;150.6
4: features 0 - 1 - 2 - 3 - 4 - 8;96.8;56.8;153.6
9: features 0 - 1 - 2 - 3 - 4 - 13;94.4;75.0;169.4
10: features 0 - 1 - 2 - 3 - 4 - 14;95.2;77.3;172.5
20: features 0 - 1 - 2 - 3 - 5 - 14;92.9;79.5;172.4
45: features 0 - 1 - 2 - 3 - 8 - 15;97.6;59.1;156.7
64: features 0 - 1 - 2 - 3 - 13 - 14;94.4;81.8;176.3
119: features 0 - 1 - 2 - 4 - 13 - 14;94.4;86.4;180.8
191: features 0 - 1 - 2 - 6 - 10 - 14;98.4;79.5;178.0
474: features 0 - 1 - 3 - 9 - 10 - 13;99.2;68.2;167.4
